In [1]:
from scipy.spatial import cKDTree
import numpy as np
import pickle
import time
from PIL import Image
from scipy.spatial import ConvexHull

In [2]:
def getEigenN(matrix):
    cov = np.cov(matrix.T)
    val, vec = np.linalg.eig(cov)
    srt = val.argsort()[::-1]
    val = val[srt]
    vec = vec[:,srt]
    if (val[2] < 0):
        val[2] = 0
    val = val/sum(val)
    return val, vec

In [3]:
def getHighIntensityRoadPoints(data, threshold, clf):
    XYZ = data[:,0:3]
    print("building KDTree")
    XYZKD = cKDTree(XYZ, leafsize=30)
    features = []
    roadIndex = []
    

    p1 = data.shape[0]/100
    p2 = 0
    #t1 = time.time()
    #t2 = time.time()
    print("go")
    
    for i in range(data.shape[0]):
        #intensity index
        if (data[i][3] >= 190):
            neighborIndex = XYZKD.query_ball_point(XYZ[i], 1)
            if len(neighborIndex) >= 3:
                neighbors = XYZ[neighborIndex]
                val, vec = getEigenN(neighbors)
                vec = vec.T
                EV1, EV2, EV3 = val[0],val[1],val[2]
                VT1, VT2, VT3 = vec[0],vec[1],vec[2]
        
                for j in range(3):
                    VT3[j] = abs(VT3[j])
                vertical = -(VT3[2]/sum(VT3) - 1)
                
                feature = [(EV1-EV2)/EV1, (EV2-EV3)/EV1, EV3/EV1, vertical]
                features.append(feature)
                roadIndex.append(i)
        
        if (i > p1):
            p1 += data.shape[0]/100
            p2 += 1
            print(str(p2)+"%")
        
        
    results = clf.predict(features)
    XYZIIndexes = []
    for i in range(len(results)):
        if (results[i] == 1):
            XYZIIndexes.append(roadIndex[i])
    return data[XYZIIndexes]

In [4]:
class RoadMarkings():
    points = None
    length = None
    width = None
    area = None
    vec = None
    val = None
    center = None
    barycenter = None
    widthCenter = None
    lengthCenter = None
    def __init__(self, points):
        if type(points) != np.ndarray:
            self.points = np.array(points)
        else:
            self.points = points
        self.__setProperties()
    
    
    def __setProperties(self):
        rec, area = self.__minimumBoundingRectangle(self.points)
        edges = []
        for i in range(1,4):
            edges.append(np.linalg.norm(rec[0]-rec[i]))
            
        edges = np.array(edges)
        srt = edges.argsort()
        edges = edges[srt]
        srt = srt.tolist()
        self.widthCenter = (rec[0] + rec[srt.index(0)+1])/2
        self.lengthCenter = (rec[0] + rec[srt.index(1)+1])/2
        self.center = (rec[0] + rec[srt.index(2)+1])/2
        
        self.width = edges[0]
        self.length = edges[1]
        self.area = area
        self.val, self.vec = self.getEigenN(self.points)
        self.barycenter = np.average(self.points, axis=0)
        
    def getEigenN(self, matrix):
        cov = np.cov(matrix.T)
        val, vec = np.linalg.eig(cov)
        srt = val.argsort()[::-1]
        val = val[srt]
        vec = vec[:,srt]

        if (val[1] < 0):
            val[1] = 0
        val = val/sum(val)
    
    
    
        return val, vec
    
    def __minimumBoundingRectangle(self, points):
        
        from scipy.ndimage.interpolation import rotate
        pi2 = np.pi/2.

        # get the convex hull for the points
        conv = ConvexHull(points)
        hull_points = points[conv.vertices]

        # calculate edge angles
        edges = np.zeros((len(hull_points)-1, 2))
        edges = hull_points[1:] - hull_points[:-1]

        angles = np.zeros((len(edges)))
        angles = np.arctan2(edges[:, 1], edges[:, 0])

        angles = np.abs(np.mod(angles, pi2))
        angles = np.unique(angles)

        # find rotation matrices
        # XXX both work
        rotations = np.vstack([
            np.cos(angles),
            np.cos(angles-pi2),
            np.cos(angles+pi2),
            np.cos(angles)]).T
        #     rotations = np.vstack([
        #         np.cos(angles),
        #         -np.sin(angles),
        #         np.sin(angles),
        #         np.cos(angles)]).T
        rotations = rotations.reshape((-1, 2, 2))

        # apply rotations to the hull
        rot_points = np.dot(rotations, hull_points.T)

        # find the bounding points
        min_x = np.nanmin(rot_points[:, 0], axis=1)
        max_x = np.nanmax(rot_points[:, 0], axis=1)
        min_y = np.nanmin(rot_points[:, 1], axis=1)
        max_y = np.nanmax(rot_points[:, 1], axis=1)

        # find the box with the best area
        areas = (max_x - min_x) * (max_y - min_y)
        best_idx = np.argmin(areas)

        # return the best box
        x1 = max_x[best_idx]
        x2 = min_x[best_idx]
        y1 = max_y[best_idx]
        y2 = min_y[best_idx]
        r = rotations[best_idx]

        rval = np.zeros((4, 2))
        rval[0] = np.dot([x1, y2], r)
        rval[1] = np.dot([x2, y2], r)
        rval[2] = np.dot([x2, y1], r)
        rval[3] = np.dot([x1, y1], r)

        return rval, conv.area

In [146]:
a =set( [1,2,3,4,5,6])
b=set([2,3,6])



In [148]:
list(a.difference(b))

[1, 4, 5]

In [250]:
class Pixel():
    #size of each pixel
    size = None
    #raw input numpy array (n * m)
    rawData = None
    #translated numpy array (3 * n), minimal value = 0
    __translatedData = None
    #min, max and range of x, y and z
    shapes = None
    #range of x, y and z
    ranges = None
    #voxelized input data (numpy array xRange * yRange * zRange)
    pixels = None
    #a list of indexes of rawData
    __pointsList = None
    #cluster data
    clusters = None
    
    crossWalk = None
    dashLine = None
    markings = None
    smallDashLine = None
    
    
    __usedClusters = None
    
    def __init__(self, rawData, size = 0.085, withKDTree = False):
        self.rawData = rawData
        self.size = size
        self.__getNewShapes()
        self.pixels = np.zeros(self.ranges[0:2])
        self.__pointsList = self.__makeList(self.pixels.size)
        self.__loadDataIndexToPointsList()
        self.__loadPointsListToPixels(threshold = 0)
        self.__cluster()
    def __getNewShapes(self):
        data = np.copy(self.rawData[:,0:3].T)
        maxXYZ = [np.amax(data[i]) for i in range(3)]
        minXYZ = [np.amin(data[i]) for i in range(3)]
        rangeXYZ = [int(maxXYZ[i]/self.size) - int(minXYZ[i]/self.size) + 1 for i in range(3)]
        
        self.ranges = rangeXYZ
        self.__translatedData = np.array([data[i] - minXYZ[i] for i in range(3)])
        self.shapes = {"xMax": maxXYZ[0], "yMax": maxXYZ[1], "zMax": maxXYZ[2],
                      "xMin": minXYZ[0], "yMin": minXYZ[1], "zMin": minXYZ[2], 
                      "x_range": rangeXYZ[0], "y_range": rangeXYZ[1], "z_range": rangeXYZ[2]}
    def __makeList(self, length):
        return [None] * length
    def __getPointsListIndexFromIJ(self, i, j):
        y_r = self.ranges[1]
        return i * y_r + j
    
    
    def __loadDataIndexToPointsList(self):
        scaledData = self.__translatedData / self.size
        
        #shape[1]: number of points
        for i in range(self.__translatedData.shape[1]):
            I, J = [int(scaledData[j][i]) for j in range(2)]
            #print([scaledData[j][i] for j in range(3)])
            index = self.__getPointsListIndexFromIJ(I, J)
            if not self.__pointsList[index]:
                self.__pointsList[index] = list()
            self.__pointsList[index].append(i)
            
    def __loadPointsListToPixels(self, threshold = 3):
        for vi in range(len(self.__pointsList)):
            if self.__pointsList[vi]:
                pointsInside = len(self.__pointsList[vi])
                if (pointsInside > threshold):
                    i, j = self.__getIJFromPointsListIndex(vi)
                    self.pixels[i][j] = pointsInside
    def __getIJFromPointsListIndex(self, index):
        y_r = self.ranges[1]
        i, j = divmod(index, y_r)
        return i, j
    def __cluster(self):
        #distance threshold
        crossDistanceMax = 30*0.065/self.size
        crossDistanceMin = 1*0.065/self.size
        dashDistanceMax = 320*0.065/self.size
        dashDistanceMin = 70*0.065/self.size
        smallDashDistanceMax = 35*0.065/self.size
        smallDashDistanceMin = 10*0.065/self.size
        
        
        #numbers
        crossWalkThreshold = 2
        dashLineThreshold = 3
        smallDashLineThreshold = 2
        
        #direction difference 
        crossDirectionThreshold = 20
        straightDirectionThreshold = 30
        smallDashDirectionThreshold = 10
        
        rows = self.ranges[0]
        cols = self.ranges[1]

        bag = []
        kernel = [(0,-2),(0,-1),(0,1),(0,2),(-1,-1),(-1,0),(-1,1),(1,-1),(1,0),(1,1),(-2,0),(2,0)]
        intensityData = np.copy(self.pixels)
        
        #---connect parts
        for i in range(2, rows - 2):
            for j in range(2, cols - 2):

                if (intensityData[i][j] >= 1):
                    cluster = []
                    near = [(i, j)]
                    while(len(near) != 0):
                        point = near.pop()
                        cluster.append(point)
                        px, py = point
                        intensityData[px][py] = 0
                        for offset in kernel:
                            tmpx = px + offset[0]
                            tmpy = py + offset[1]
                            if (tmpx >= rows) or (tmpy >= cols):
                                continue
                            if intensityData[tmpx][tmpy] >= 1:
                                intensityData[tmpx][tmpy] = 0
                                near.append(tuple((tmpx, tmpy)))

                    bag.append(cluster)
        #connect parts---
        
        #---make mark
        self.clusters = []
        for i in range(len(bag)):
            if (len(bag[i]) > 20):
                RM = RoadMarkings(bag[i])
                self.clusters.append(RM)
        #make mark---
        
        
        #connect marks---

        dashLine = []
        crossWalk = []
        
        centers = np.array([rm.center for rm in self.clusters])
        RMKD = cKDTree(centers ,leafsize = 30)
        
        
        
        self.__usedClusters = [False] * len(self.clusters)
        
        #cross walk
        discovered = [False] * len(self.clusters)
        
        for i in range(len(discovered)):
            if not discovered[i]:
                discovered[i] = True
                pending = [i]
                group = []
                while len(pending) > 0:
                    current = pending.pop()
                    if 20*0.065/self.size<self.clusters[current].length<90*0.065/self.size:
                        if 1.5*0.065/self.size<self.clusters[current].width <10*0.065/self.size:
                            #is line
                            group.append(current)
                            neighborIndex = set(RMKD.query_ball_point(centers[current], crossDistanceMax))
                            without = set(RMKD.query_ball_point(centers[current], crossDistanceMin))
                            neighborIndex = list(neighborIndex.difference(without))
                            
                            for j in range(0, len(neighborIndex)):
                                cluID = neighborIndex[j]
                                if not discovered[cluID]:
                                    if 20*0.065/self.size<self.clusters[cluID].length<90*0.065/self.size:
                                        if 1.5*0.065/self.size<self.clusters[cluID].width <10*0.065/self.size:
                                            diff = (self.clusters[current].center - self.clusters[cluID].center)/   ((np.array(self.clusters[cluID].vec.T[1]) + np.array(self.clusters[current].vec.T[1]))/2)
                                            #print("hmm",diff)
                                            if abs(diff[0]-diff[1]) <= crossDirectionThreshold:
                                                pending.append(cluID)
                                                discovered[cluID] = True
                                else:
                                    continue
                if len(group) >= crossWalkThreshold:
                    crossWalk.append(group)
                    for idx in group:
                        self.__usedClusters[idx] = True
            else:
                pass
            
        #dash line
        discovered = np.array(self.__usedClusters)
        
        for i in range(len(discovered)):
            if not discovered[i]:
                discovered[i] = True
                pending = [i]
                group = []
                while len(pending) > 0:
                    current = pending.pop()
                    if 20*0.065/self.size<self.clusters[current].length<60*0.065/self.size:
                        if 1.5*0.065/self.size<self.clusters[current].width <10*0.065/self.size:
                            #is line
                            group.append(current)
                            neighborIndex = set(RMKD.query_ball_point(centers[current], dashDistanceMax))
                            without = set(RMKD.query_ball_point(centers[current], dashDistanceMin))
                            neighborIndex = list(neighborIndex.difference(without))
                            for j in range(0, len(neighborIndex)):
                                cluID = neighborIndex[j]
                                if not discovered[cluID]:
                                    if 20*0.065/self.size<self.clusters[cluID].length<60*0.065/self.size:
                                        if 1.5*0.065/self.size<self.clusters[cluID].width <10*0.065/self.size:
                                            diff = (self.clusters[current].center - self.clusters[cluID].center)/   ((np.array(self.clusters[cluID].vec.T[0]) + np.array(self.clusters[current].vec.T[0]))/2)
                                            if abs(diff[0]-diff[1]) <= straightDirectionThreshold:
                                                pending.append(cluID)
                                                discovered[cluID] = True
                                else:
                                    continue
                if len(group) >= dashLineThreshold:
                    dashLine.append(group)
                    for idx in group:
                        self.__usedClusters[idx] = True
            else:
                pass

        
        
        #markings
        markings = []
        discovered = np.array(self.__usedClusters)
        for i in range(len(discovered)):
            if not discovered[i]:
                discovered[i] = True
                if 50*0.065/self.size<self.clusters[i].length<130*0.065/self.size:
                    if 8*0.065/self.size<self.clusters[i].width <20*0.065/self.size:
                        markings.append([i])
                        self.__usedClusters[i] = True
        
        #smallDashLine
        smallDashLine = []
        discovered = np.array(self.__usedClusters)
        for i in range(len(discovered)):
            if not discovered[i]:
                discovered[i] = True
                pending = [i]
                group = []
                while len(pending) > 0:
                    current = pending.pop()
                    if 5*0.085/self.size<self.clusters[current].length<14*0.085/self.size:
                        if 1.75*0.085/self.size<self.clusters[current].width <3.25*0.085/self.size:
                            #is line
                            group.append(current)
                            neighborIndex = set(RMKD.query_ball_point(centers[current], smallDashDistanceMax))
                            without = set(RMKD.query_ball_point(centers[current], smallDashDistanceMin))
                            neighborIndex = list(neighborIndex.difference(without))
                            for j in range(0, len(neighborIndex)):
                                cluID = neighborIndex[j]
                                if not discovered[cluID]:
                                    if 5*0.085/self.size<self.clusters[cluID].length<14*0.085/self.size:
                                        if 1.75*0.085/self.size<self.clusters[cluID].width <3.25*0.085/self.size:
                                            diff = (self.clusters[current].center - self.clusters[cluID].center)/   ((np.array(self.clusters[cluID].vec.T[0]) + np.array(self.clusters[current].vec.T[0]))/2)
                                            if abs(diff[0]-diff[1]) <= smallDashDirectionThreshold:
                                                pending.append(cluID)
                                                discovered[cluID] = True
                                                print(self.clusters[current].val[0])
                                else:
                                    continue
                if len(group) >= smallDashLineThreshold:
                    smallDashLine.append(group)
                    for idx in group:
                        self.__usedClusters[idx] = True
            else:
                pass
        
        
        
        self.crossWalk = crossWalk
        self.dashLine = dashLine
        self.markings = markings
        self.smallDashLine = smallDashLine
        
        
        
        
        #---connect marks
        

In [ ]:
def point_in_hull(point, hull, tolerance=1e-12):
    return all(
        (np.dot(eq[:-1], point) + eq[-1] <= tolerance)
        for eq in hull.equations)

In [6]:
def gray_2_rgb(img):
    
    new_img = np.zeros((img.shape[0],img.shape[1],3))
        
    for i in range(img.shape[0]):
        for k in range(img.shape[1]):
            if img[i][k] >= 1:
                new_img[i][k] = np.array([255,255,255])
    
    new_img = new_img.astype('uint8')
    
    return new_img


In [209]:
XYZI = np.genfromtxt("withGround06.csv", delimiter=",", usecols=(0,1,2,3))
with open('roadClassifier.pickle', 'rb') as f:
    clf = pickle.load(f)
highIntensityRoadPoints = getHighIntensityRoadPoints(XYZI, 190, clf)

building KDTree
go
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%


In [212]:
a = highIntensityRoadPoints.tolist()
import csv as c
with open("QQ.csv","w") as f:
    writer = c.writer(f)
    for i in range(len(a)):
        writer.writerow(a[i])

In [210]:
#save
with open("withGround06_highIntensityRoadPoints.pickle", "wb") as f:
    pickle.dump(highIntensityRoadPoints,f)

In [175]:
with open("withGround01_highIntensityRoadPoints.pickle", "rb") as f:
    highIntensityRoadPoints = pickle.load(f)

In [251]:
b = Pixel(highIntensityRoadPoints)
q = b.clusters

0.9450071476982246
0.9637293294433766
0.9541220766327253
0.9657096144200432
0.9599622242158715
0.9455343678369716
0.948460251842527
0.9685538969678346
0.9614372023191882
0.9604713784180046
0.9547852388656614
0.954198845277031
0.9582965649914981
0.9500970097103609
0.9571646779127737
0.9545167196064163
0.9149356439545278
0.9473544422357595
0.9528467980159698
0.940687774893138
0.9384925253227174
0.9406174204813842
0.9161227465459711
0.9234527728247514
0.9399484898644807
0.9517227710022174
0.9594497646232951
0.9666436643190776
0.9689245239418857
0.9637618588352586
0.9591128497263232
0.9563573094744464
0.945809422911115
0.9401399591473436
0.9604402234247299
0.9346767852045172


In [239]:
b.markings

[[13], [22], [24], [27], [30], [35], [40], [94]]

In [252]:
b.crossWalk


[[8, 14, 19], [11, 17], [39, 44]]

In [241]:
b.dashLine

[]

In [242]:
b.smallDashLine

[[18, 21, 25, 29, 31, 36, 41, 47, 49, 52, 54, 56, 58, 61, 64],
 [37, 38, 43, 45, 48, 50, 51, 53, 55, 57, 59, 63, 67],
 [60, 62],
 [70, 74, 78, 82],
 [73, 77, 81, 83, 85, 87],
 [75, 79]]

In [229]:
for i in [73, 79, 75, 77, 81, 84, 83, 85, 87]:
    print(b.clusters[i].vec)

[[ 0.6975235  -0.71656191]
 [-0.71656191 -0.6975235 ]]
[[ 0.87632279 -0.48172436]
 [ 0.48172436  0.87632279]]
[[ 0.75365112 -0.65727467]
 [ 0.65727467  0.75365112]]
[[ 0.71018356  0.70401656]
 [-0.70401656  0.71018356]]
[[ 0.75319888  0.65779286]
 [-0.65779286  0.75319888]]
[[ 0.86451597 -0.50260535]
 [ 0.50260535  0.86451597]]
[[ 0.79829934  0.60226087]
 [-0.60226087  0.79829934]]
[[ 0.82088765  0.57108972]
 [-0.57108972  0.82088765]]
[[ 0.89013956  0.455688  ]
 [-0.455688    0.89013956]]


In [195]:
b.clusters[1].length

10.43551627855565

In [189]:
for i in range(len(b._Pixel__usedClusters)):
    if not (b._Pixel__usedClusters[i]):
        print(b.clusters[i].length, b.clusters[i].width)


26.27327490232734 7.6300469579361625
10.43551627855565 2.5298221281347026
10.666987174671934 2.852798895551797
7.060180864974622 5.883484054145525
9.986876634765906 2.432700718725017
10.606601717798219 2.1213203435596526
8.48528137423857 2.8284271247462103
10.261953630166746 2.773500981126134
10.28591269649902 2.683281572999754
8.944271909999133 2.683281572999735
8.731282501307984 2.667891875399649
8.538149682454593 2.21359436211787
8.854377448471471 2.2135943621178655
7.452413135251007 2.5495097567963922
9.217458351494628 2.3533936216581917
7.800000000000011 2.4000000000000283
7.7781745930520225 2.1213203435596424


In [26]:
a =b.clusters[6]
a.length,a.width

(77.04800140527941, 12.248119788610996)

In [243]:
b.smallDashLine

[[18, 21, 25, 29, 31, 36, 41, 47, 49, 52, 54, 56, 58, 61, 64],
 [37, 38, 43, 45, 48, 50, 51, 53, 55, 57, 59, 63, 67],
 [60, 62],
 [70, 74, 78, 82],
 [73, 77, 81, 83, 85, 87],
 [75, 79]]

In [244]:
tmp=[]
for i in range(len(b.smallDashLine)):
    with open(str(i)+"smallD.csv","w",newline="") as f:
        wr = c.writer(f)
        for idx in b.smallDashLine[i]:
            a = b.clusters[idx].points.tolist()
            for aa in a:
                wr.writerow(aa)

In [246]:
tmp=[]
for i in range(len(b._Pixel__usedClusters)):
    if not (b._Pixel__usedClusters[i]):
        with open(str(i)+"remain.csv","w",newline="") as f:
            wr = c.writer(f)
            for pt in b.clusters[i].points:
                wr.writerow(pt.tolist())

In [254]:
tmp=[]
for i in range(len(b._Pixel__usedClusters)):
    if not (b._Pixel__usedClusters[i]):
        for pt in b.clusters[i].points:
            tmp.append(pt.tolist())
import csv as c
with open("remain.csv","w",newline="") as f:
    wr = c.writer(f)
    for i in range(len(tmp)):
        wr.writerow(tmp[i])

In [219]:
#save markings
tmp = []
for i in b.markings:
    for j in i:
        tmp += b.clusters[j].points.tolist()
import csv as c
with open("tr3markings.csv","w")as f:
    wr = c.writer(f)
    for i in range(len(tmp)):
        wr.writerow(tmp[i])

In [253]:
#save crosswalk
tmp = []
for i in b.crossWalk:
    for j in i:
        tmp += b.clusters[j].points.tolist()
import csv as c
with open("tr3cross.csv","w")as f:
    wr = c.writer(f)
    for i in range(len(tmp)):
        wr.writerow(tmp[i])

In [221]:
#save dashline
tmp = []
for i in b.dashLine:
    for j in i:
        tmp += b.clusters[j].points.tolist()
import csv as c
with open("tr3straight.csv","w")as f:
    wr = c.writer(f)
    for i in range(len(tmp)):
        wr.writerow(tmp[i])

In [222]:
#save smalldashline
tmp = []
for i in b.smallDashLine:
    for j in i:
        tmp += b.clusters[j].points.tolist()
import csv as c
with open("tr3smallDash.csv","w")as f:
    wr = c.writer(f)
    for i in range(len(tmp)):
        wr.writerow(tmp[i])

In [143]:
#save roadmarkings
tmp = []
for rm in b.clusters:
    for i in range(len(rm.points)):
        tmp.append(rm.points[i])
import csv as c
with open("rms.csv","w")as f:
    wr = c.writer(f)
    for i in range(len(tmp)):
        wr.writerow(tmp[i])
    


In [179]:
rgb = gray_2_rgb(b.pixels)
Image.fromarray(rgb).save("QQ.png")

In [94]:
tmp = []
cc = 0
for roadmark in q:
    cc+= 1
    if 20<roadmark.length <60:
        
        if 1.5<roadmark.width <10:
            r = roadmark.points.tolist()
            for point in r:
                point += [0,255,255,0]
            tmp += r
#     if 35<roadmark.length <60:
#         if 5<roadmark.width <10:
#             r = roadmark.points.tolist()
#             for point in r:
#                 point += [0,110,255,135]
#             tmp += r
            

43.909761034352705
49.798727843165715
45.10671108178235


In [92]:
import csv as c
with open("tr.csv","w")as f:
    wr = c.writer(f)
    for i in range(len(tmp)):
        wr.writerow(tmp[i])

In [54]:
#straight
for i in range(3):
    print(q[i].vec)

[[ 0.98427515 -0.17664212]
 [ 0.17664212  0.98427515]]
[[ 0.98302295 -0.18348265]
 [ 0.18348265  0.98302295]]
[[ 0.98112743 -0.19336229]
 [ 0.19336229  0.98112743]]


In [65]:
#cross
for i in range(3):
    print(q[i].vec)

[[ 0.15497479 -0.98791842]
 [-0.98791842 -0.15497479]]
[[ 0.16243637 -0.98671902]
 [-0.98671902 -0.16243637]]
[[ 0.14934165 -0.98878566]
 [-0.98878566 -0.14934165]]


In [66]:
for i in range(3):
    print(q[i].center)

[ 6.14563107 21.49838188]
[24.04040404 24.51515152]
[42.36567164 27.58955224]
